In [ ]:
# same analysis - but with scrubbingb

In [1]:
## Use this box if you want MDSL
# using MSDL atlas - Can choose different atlas or different nodes (using ICA or something else)
from nilearn import datasets
atlas = datasets.fetch_atlas_msdl()

# Loading atlas image stored in 'maps'
atlas_filename = atlas['maps']
# Loading atlas data stored in 'labels'
#labels_img=yeo['thick_17']
labels =  atlas['labels']
coords = atlas.region_coords# grab center coordinates for atlas labels

/gpfs/ysm/project/oad4/conda_envs/py37_dev/lib/python3.6/site-packages/numpy/lib/npyio.py:2322: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [10]:
import os
import numpy as np

# methods
def removeVars (confoundFile):
    # this method takes the csv regressors file (from fmriPrep) and chooses a few to confound. You can change those few
    import pandas as pd
    confound = pd.read_csv(confoundFile,sep="\t", na_values="n/a")
    finalConf = confound[['csf', 'white_matter', 'framewise_displacement', 'std_dvars',
                          'a_comp_cor_00', 'a_comp_cor_01',	'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04', 
                        'a_comp_cor_05', 'trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']] # can add 'global_signal' also
     # change NaN of FD to zero
    finalConf = np.array(finalConf)
    finalConf[0,2] = 0 # change first FD from NaN to zero
    finalConf[0,3] = 0 # change first DVARS from NaN to zero
    return finalConf

In [23]:
# build method for creating time series for subjects
def timeSeries(func_files, confound_files):
    total_subjects = [] # creating an empty array that will hold all subjects matrix 
    # This function needs a masker object that will be defined outside the function
    for func_file, confound_file in zip(func_files, confound_files):
        confoundClean = removeVars(confound_file)
        confoundArray = scrubb(confoundClean, FDvalue=0.8, dvarValue=1.5) # adding scrubb
        time_series = masker.fit_transform(func_file, confounds=confoundArray)
        #time_series = extractor.fit_transform(func_file, confounds=confoundArray)
        #masker.fit_transform(func_file, confoundArray)
        total_subjects.append(time_series)
    return total_subjects

# contrasting two timePoints
def contFuncs(time_series1, time_series2):
    twoMinusOneMat = []
    for scanMatrix, scanMatrix2 in zip(time_series1, time_series2):
        a = scanMatrix2 - scanMatrix
        twoMinusOneMat.append(a)
    return np.array(twoMinusOneMat)

import numpy as np
from nilearn import plotting

# create correlation matrix per subject
def createCorMat(time_series):
    # create correlation matrix for each subject
    fullMatrix = []
    for time_s in time_series:
        correlation_matrix = correlation_measure.fit_transform([time_s])[0]
        fullMatrix.append(correlation_matrix)
    return fullMatrix

# create connecotme graph per subject
def connectome_graph (fullMatrix):
    # here it is set to threshold 1%
    for matrix in fullMatrix:
        plotting.plot_connectome(matrix, coords,
                             edge_threshold="99%", colorbar=True)
        plotting.show()

In [24]:
# writing the actual scrubbing method in which we will eventually add a column of 0 or 1 for scrubbing. (1 = this TR should be scrubbed)
def scrubb(param, FDvalue, dvarValue):
    # take parameter array
    fd = param[:,2] # take only FD column
    dvars = param[:,3] # take only DVARS std columns
    # extract DVARS and FD
    # run through all TRs and check for changes
    x = [] # empty array
    for i in fd:
        if i>=FDvalue:
            print ("FD outlier")
            x.append(1)
        else:
            x.append(0)     
    for z,n in enumerate(dvars):
        if n >= dvarValue:
            print ("DVARS outlier")
            x[z]=1
        else:
            if x[z]==0:
                x[z]=0
            else:
                x[z] = 1
    # create a vector of 0 or 1
    # return full array with added column
    print(np.array(x).shape)
    print (x)
    print(param.shape)
    fullParam = np.array(param)
    fullParam = np.column_stack([param, x])

    return fullParam

In [25]:
# Here you set the specific methods for masking and correlation. Please see Nilearn website for more info.

from nilearn.input_data import NiftiMapsMasker

# in this mask we standardize the values, so mean is 0 and between -1 to 1
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, smoothing_fwhm = 6,
                         memory="/home/oad4/scratch60/nilearn",high_pass=.01 , low_pass = .1, t_r=1, verbose=5)


from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='partial correlation') # can choose partial - it might be better


In [26]:
# now we call subjcets
# and start the real analysis
subList =  ['008','1293','1307','1322','1339','1343','1387','1223']
midSubList = ['1253','1263','1351','1364','1369','1390','1403']

# these two functions take subject list and session number (in string) and return func file list and confound file list
def fileList(subjects, session):
    func_files = ['/home/oad4/scratch60/kpeOutput/fmriprep/sub-%s/ses-%s/func/sub-%s_ses-%s_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' % (sub,session,sub,session) for sub in subjects]
    return func_files

def confList(subjects, session):
    confound_files = ['/home/oad4/scratch60/kpeOutput/fmriprep/sub-%s/ses-%s/func/sub-%s_ses-%s_task-rest_desc-confounds_regressors.tsv' % (sub,session,sub,session) for sub in subjects]
    return confound_files


In [27]:
# now we call for the functions for each set.
# for every time line we want to run time series and then contrast between the times
ket1_series = timeSeries(func_files=fileList(subList,'1'), confound_files=confList(subList, '1'))
ket2_series = timeSeries(func_files=fileList(subList,'2'), confound_files=confList(subList, '2'))
ket3_series = timeSeries(func_files=fileList(subList,'3'), confound_files=confList(subList, '3'))

(580,)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

/gpfs/ysm/project/oad4/conda_envs/py37_dev/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:84: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/home/oad4/scratch60/nilearn'", use "location='/home/oad4/scratch60/nilearn'" instead.
  memory = Memory(cachedir=cache_dir, verbose=verbose)
/gpfs/ysm/project/oad4/conda_envs/py37_dev/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:301: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.memory_level == 0 and self.memory.cachedir is not None:
/gpfs/ysm/project/oad4/conda_envs/py37_dev/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:302: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  warnings.warn("memory_level is curre

________________________________________resample_img cache loaded - 2.9s, 0.0min
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract('/home/oad4/scratch60/kpeOutput/fmriprep/sub-008/ses-1/func/sub-008_ses-1_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', 
{ 'allow_overlap': True,
  'detrend': False,
  'dtype': None,
  'high_pass': 0.01,
  'low_pass': 0.1,
  'maps_img': '/home/oad4/nilearn_data/msdl_atlas/MSDL_rois/msdl_rois.nii',
  'mask_img': None,
  'smoothing_fwhm': 6,
  'standardize': True,
  't_r': 1,
  'target_affine': None,
  'target_shape': None}, confounds=array([[5222.752157, ...,    0.      ],
       ...,
       [5127.638084, ...,    0.      ]]), dtype=None, memory=Memory(location=/home/oad4/scratch60/nilearn/joblib), memory_level=1, verbose=5)
[NiftiMapsMasker.transform_single_imgs] Loading data from /home/oad4/scratch60/kpeOutput/fmriprep/sub

/gpfs/ysm/project/oad4/conda_envs/py37_dev/lib/python3.6/site-packages/nilearn/input_data/nifti_maps_masker.py:326: UserWarning: Persisting input arguments took 1.14s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  verbose=self.verbose)


______________________________________________filter_and_extract - 87.7s, 1.5min
(580,)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

/gpfs/ysm/project/oad4/conda_envs/py37_dev/lib/python3.6/site-packages/nilearn/input_data/nifti_maps_masker.py:326: UserWarning: Persisting input arguments took 1.14s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  verbose=self.verbose)


______________________________________________filter_and_extract - 88.0s, 1.5min
(580,)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

/gpfs/ysm/project/oad4/conda_envs/py37_dev/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract('/home/oad4/scratch60/kpeOutput/fmriprep/sub-1223/ses-3/func/sub-1223_ses-3_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', 
{ 'allow_overlap': True,
  'detrend': False,
  'dtype': None,
  'high_pass': 0.01,
  'low_pass': 0.1,
  'maps_img': '/home/oad4/nilearn_data/msdl_atlas/MSDL_rois/msdl_rois.nii',
  'mask_img': None,
  'smoothing_fwhm': 6,
  'standardize': True,
  't_r': 1,
  'target_affine': None,
  'target_shape': None}, confounds=array([[5331.850292, ...,    0.      ],
       ...,
       [5183.18    , ...,    0.      ]]), dtype=None, memory=Memory(location=/home/oad4/scratch60/nilearn/joblib), memory_level=1, verbose=5)
[NiftiMapsMasker.transform_single_imgs] Loading data from /home/oad4/scratch60/kpeOutput/fmriprep/sub-1223/ses-3/func/sub-1223_ses-3_task-rest_space-MNI152NLin2009cAsym_desc-prepro

/gpfs/ysm/project/oad4/conda_envs/py37_dev/lib/python3.6/site-packages/nilearn/input_data/nifti_maps_masker.py:326: UserWarning: Persisting input arguments took 1.14s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  verbose=self.verbose)


In [28]:
# build correlation matrix for each time point
ket1_corr = createCorMat(time_series=ket1_series)
ket2_corr = createCorMat(ket2_series)
ket3_corr = createCorMat(ket3_series)

In [29]:
# rearrange the array to N,N,subnumber for NBS
ket1Reshape = np.moveaxis(np.array(ket1_corr), 0,-1)
print(ket1Reshape.shape)
ket2Reshape = np.moveaxis(np.array(ket2_corr), 0,-1)
ket3Reshape = np.moveaxis(np.array(ket3_corr), 0,-1)


(39, 39, 8)


In [39]:
# now we can run NBS
# NBS is taken from: https://github.com/aestrivex/bctpy, can be installed using pip (pip install bctpy)
from bct import nbs
pval, adj, _ = nbs.nbs_bct(ket1Reshape, ket3Reshape, thresh=2.5, tail='both',k=1500, paired=True, verbose = True)

max component size is 38
estimating null distribution with 1500 permutations
permutation 0 of 1500.  Permutation max is 25.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 1 of 1500.  Permutation max is 31.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 2 of 1500.  Permutation max is 32.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 3 of 1500.  Permutation max is 28.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 4 of 1500.  Permutation max is 19.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 5 of 1500.  Permutation max is 27.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 6 of 1500.  Permutation max is 23.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 7 of 1500.  Permutation max is 27.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 8 of 1500.  Permutation max is 28.0.  Observed max is 38.0.  P-val estimate is 0.000
permutation 9 of 1500.  Permutation max is 5.0.  O

permutation 88 of 1500.  Permutation max is 29.0.  Observed max is 38.0.  P-val estimate is 0.146
permutation 89 of 1500.  Permutation max is 36.0.  Observed max is 38.0.  P-val estimate is 0.144
permutation 90 of 1500.  Permutation max is 19.0.  Observed max is 38.0.  P-val estimate is 0.143
permutation 91 of 1500.  Permutation max is 25.0.  Observed max is 38.0.  P-val estimate is 0.141
permutation 92 of 1500.  Permutation max is 15.0.  Observed max is 38.0.  P-val estimate is 0.140
permutation 93 of 1500.  Permutation max is 21.0.  Observed max is 38.0.  P-val estimate is 0.138
permutation 94 of 1500.  Permutation max is 29.0.  Observed max is 38.0.  P-val estimate is 0.137
permutation 95 of 1500.  Permutation max is 37.0.  Observed max is 38.0.  P-val estimate is 0.135
permutation 96 of 1500.  Permutation max is 31.0.  Observed max is 38.0.  P-val estimate is 0.134
permutation 97 of 1500.  Permutation max is 15.0.  Observed max is 38.0.  P-val estimate is 0.133
permutation 98 of 15

permutation 172 of 1500.  Permutation max is 36.0.  Observed max is 38.0.  P-val estimate is 0.116
permutation 173 of 1500.  Permutation max is 46.0.  Observed max is 38.0.  P-val estimate is 0.121
permutation 174 of 1500.  Permutation max is 15.0.  Observed max is 38.0.  P-val estimate is 0.120
permutation 175 of 1500.  Permutation max is 26.0.  Observed max is 38.0.  P-val estimate is 0.119
permutation 176 of 1500.  Permutation max is 26.0.  Observed max is 38.0.  P-val estimate is 0.119
permutation 177 of 1500.  Permutation max is 16.0.  Observed max is 38.0.  P-val estimate is 0.118
permutation 178 of 1500.  Permutation max is 10.0.  Observed max is 38.0.  P-val estimate is 0.117
permutation 179 of 1500.  Permutation max is 37.0.  Observed max is 38.0.  P-val estimate is 0.117
permutation 180 of 1500.  Permutation max is 28.0.  Observed max is 38.0.  P-val estimate is 0.116
permutation 181 of 1500.  Permutation max is 10.0.  Observed max is 38.0.  P-val estimate is 0.115
permutatio

permutation 256 of 1500.  Permutation max is 31.0.  Observed max is 38.0.  P-val estimate is 0.117
permutation 257 of 1500.  Permutation max is 22.0.  Observed max is 38.0.  P-val estimate is 0.116
permutation 258 of 1500.  Permutation max is 25.0.  Observed max is 38.0.  P-val estimate is 0.116
permutation 259 of 1500.  Permutation max is 24.0.  Observed max is 38.0.  P-val estimate is 0.115
permutation 260 of 1500.  Permutation max is 33.0.  Observed max is 38.0.  P-val estimate is 0.115
permutation 261 of 1500.  Permutation max is 39.0.  Observed max is 38.0.  P-val estimate is 0.118
permutation 262 of 1500.  Permutation max is 36.0.  Observed max is 38.0.  P-val estimate is 0.118
permutation 263 of 1500.  Permutation max is 39.0.  Observed max is 38.0.  P-val estimate is 0.121
permutation 264 of 1500.  Permutation max is 37.0.  Observed max is 38.0.  P-val estimate is 0.121
permutation 265 of 1500.  Permutation max is 19.0.  Observed max is 38.0.  P-val estimate is 0.120
permutatio

permutation 340 of 1500.  Permutation max is 9.0.  Observed max is 38.0.  P-val estimate is 0.132
permutation 341 of 1500.  Permutation max is 11.0.  Observed max is 38.0.  P-val estimate is 0.132
permutation 342 of 1500.  Permutation max is 24.0.  Observed max is 38.0.  P-val estimate is 0.131
permutation 343 of 1500.  Permutation max is 43.0.  Observed max is 38.0.  P-val estimate is 0.134
permutation 344 of 1500.  Permutation max is 31.0.  Observed max is 38.0.  P-val estimate is 0.133
permutation 345 of 1500.  Permutation max is 25.0.  Observed max is 38.0.  P-val estimate is 0.133
permutation 346 of 1500.  Permutation max is 20.0.  Observed max is 38.0.  P-val estimate is 0.133
permutation 347 of 1500.  Permutation max is 5.0.  Observed max is 38.0.  P-val estimate is 0.132
permutation 348 of 1500.  Permutation max is 21.0.  Observed max is 38.0.  P-val estimate is 0.132
permutation 349 of 1500.  Permutation max is 9.0.  Observed max is 38.0.  P-val estimate is 0.131
permutation 3

...

permutation 1096 of 1500.  Permutation max is 19.0.  Observed max is 38.0.  P-val estimate is 0.129
permutation 1097 of 1500.  Permutation max is 35.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1098 of 1500.  Permutation max is 25.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1099 of 1500.  Permutation max is 31.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1100 of 1500.  Permutation max is 9.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1101 of 1500.  Permutation max is 10.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1102 of 1500.  Permutation max is 39.0.  Observed max is 38.0.  P-val estimate is 0.129
permutation 1103 of 1500.  Permutation max is 21.0.  Observed max is 38.0.  P-val estimate is 0.129
permutation 1104 of 1500.  Permutation max is 19.0.  Observed max is 38.0.  P-val estimate is 0.129
permutation 1105 of 1500.  Permutation max is 26.0.  Observed max is 38.0.  P-val estimate is 0.128
p

permutation 1180 of 1500.  Permutation max is 10.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1181 of 1500.  Permutation max is 23.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1182 of 1500.  Permutation max is 26.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1183 of 1500.  Permutation max is 25.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1184 of 1500.  Permutation max is 40.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1185 of 1500.  Permutation max is 20.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1186 of 1500.  Permutation max is 37.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1187 of 1500.  Permutation max is 22.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1188 of 1500.  Permutation max is 43.0.  Observed max is 38.0.  P-val estimate is 0.129
permutation 1189 of 1500.  Permutation max is 36.0.  Observed max is 38.0.  P-val estimate is 0.129


permutation 1264 of 1500.  Permutation max is 23.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1265 of 1500.  Permutation max is 25.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1266 of 1500.  Permutation max is 27.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1267 of 1500.  Permutation max is 32.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1268 of 1500.  Permutation max is 25.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1269 of 1500.  Permutation max is 32.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1270 of 1500.  Permutation max is 26.0.  Observed max is 38.0.  P-val estimate is 0.127
permutation 1271 of 1500.  Permutation max is 41.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1272 of 1500.  Permutation max is 35.0.  Observed max is 38.0.  P-val estimate is 0.128
permutation 1273 of 1500.  Permutation max is 38.0.  Observed max is 38.0.  P-val estimate is 0.129


permutation 1348 of 1500.  Permutation max is 28.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1349 of 1500.  Permutation max is 21.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1350 of 1500.  Permutation max is 17.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1351 of 1500.  Permutation max is 27.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1352 of 1500.  Permutation max is 28.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1353 of 1500.  Permutation max is 11.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1354 of 1500.  Permutation max is 34.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1355 of 1500.  Permutation max is 23.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1356 of 1500.  Permutation max is 10.0.  Observed max is 38.0.  P-val estimate is 0.125
permutation 1357 of 1500.  Permutation max is 43.0.  Observed max is 38.0.  P-val estimate is 0.125


permutation 1432 of 1500.  Permutation max is 34.0.  Observed max is 38.0.  P-val estimate is 0.121
permutation 1433 of 1500.  Permutation max is 7.0.  Observed max is 38.0.  P-val estimate is 0.121
permutation 1434 of 1500.  Permutation max is 31.0.  Observed max is 38.0.  P-val estimate is 0.121
permutation 1435 of 1500.  Permutation max is 32.0.  Observed max is 38.0.  P-val estimate is 0.121
permutation 1436 of 1500.  Permutation max is 40.0.  Observed max is 38.0.  P-val estimate is 0.122
permutation 1437 of 1500.  Permutation max is 20.0.  Observed max is 38.0.  P-val estimate is 0.122
permutation 1438 of 1500.  Permutation max is 28.0.  Observed max is 38.0.  P-val estimate is 0.122
permutation 1439 of 1500.  Permutation max is 17.0.  Observed max is 38.0.  P-val estimate is 0.122
permutation 1440 of 1500.  Permutation max is 32.0.  Observed max is 38.0.  P-val estimate is 0.121
permutation 1441 of 1500.  Permutation max is 23.0.  Observed max is 38.0.  P-val estimate is 0.121
p

In [40]:
print(pval.shape)
print (pval)

(1,)
[0.12333333]
